In [0]:
# FAZ CONEXÃO COM O S3 DA AWS NA NUVEM 

access_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
secret_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

In [0]:
# CONFIGURA, PARA O HADOOP, A REGIÃO DE ACESSO PARA O S3 DA AWS NA NUVEM

aws_region = "us-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

In [0]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD = sc.textFile("s3a://arq-ipca-processeds3/")
myRDD2 = sc.textFile("s3a://arquivos-pmc-processeds3-uf/")
myRDD3 = sc.textFile("s3a://arquivos-pmc-processeds3-catcom/")

myRDD.count()
dbutils.fs.ls("s3a://arq-ipca-processeds3/")

Out[45]: [FileInfo(path='s3a://arq-ipca-processeds3/ipca.pq', name='ipca.pq', size=8920, modificationTime=1663099411000)]

In [0]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD2.count()
dbutils.fs.ls("s3a://arquivos-pmc-processeds3-uf/")

Out[35]: [FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202201_02.pq', name='PercUF_pmc_202201_02.pq', size=5035, modificationTime=1663247975000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202201_03.pq', name='PercUF_pmc_202201_03.pq', size=5032, modificationTime=1663247979000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202201_07.pq', name='PercUF_pmc_202201_07.pq', size=5041, modificationTime=1663247992000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202201_08.pq', name='PercUF_pmc_202201_08.pq', size=5021, modificationTime=1663247996000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202202_02.pq', name='PercUF_pmc_202202_02.pq', size=5035, modificationTime=1663248014000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202202_03.pq', name='PercUF_pmc_202202_03.pq', size=5021, modificationTime=1663248018000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-uf/PercUF_pmc_202202_07.pq', nam

In [0]:
# LISTA OS ARQUIVOS DO BUCKET

myRDD3.count()
dbutils.fs.ls("s3a://arquivos-pmc-processeds3-catcom/")

Out[36]: [FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202201_01.pq', name='PercCAT_COMERCIO_pmc_202201_01.pq', size=5461, modificationTime=1663247971000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202201_05.pq', name='PercCAT_COMERCIO_pmc_202201_05.pq', size=5384, modificationTime=1663247984000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202201_06.pq', name='PercCAT_COMERCIO_pmc_202201_06.pq', size=5466, modificationTime=1663247988000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202201_10.pq', name='PercCAT_COMERCIO_pmc_202201_10.pq', size=5378, modificationTime=1663248001000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202202_01.pq', name='PercCAT_COMERCIO_pmc_202202_01.pq', size=5471, modificationTime=1663248011000),
 FileInfo(path='s3a://arquivos-pmc-processeds3-catcom/PercCAT_COMERCIO_pmc_202202_05.pq', name='PercCAT_COMERCI

In [0]:
# LENDO O SCHEMA DOS ARQUIVOS PARQUET DE CADA BUCKET

spark.read.format("parquet").load("s3a://arq-ipca-processeds3/")
spark.read.format("parquet").load("s3a://arquivos-pmc-processeds3-uf/")
spark.read.format("parquet").load("s3a://arquivos-pmc-processeds3-catcom/")

Out[37]: DataFrame[registro: bigint, classe_comercio: string, ano_mes: string, ano: string, mes: string, m3mensal: string]

In [0]:
# EXTRAINDO OS DADOS DE IPCA DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK

dfipca = spark.sql("SELECT ano, mes, avg(perc) as perc \
                            FROM parquet.`s3a://arq-ipca-processeds3/` \
                            where mes < 13 \
                            group by ano, mes order by ano, mes")

from pyspark.sql.functions import lit
#dfipca = dfipca.withColumn("id", lit("ipca"))
dfipca.show()

+----+---+----+
| ano|mes|perc|
+----+---+----+
|1980|  1|6.62|
|1980| 10|9.48|
|1980| 11|6.67|
|1980| 12|6.61|
|1980|  2|4.62|
|1980|  3|6.04|
|1980|  4|5.29|
|1980|  5| 5.7|
|1980|  6|5.31|
|1980|  7|5.55|
|1980|  8|4.95|
|1980|  9|4.23|
|1981|  1|6.84|
|1981| 10|5.08|
|1981| 11|5.27|
|1981| 12|5.93|
|1981|  2| 6.4|
|1981|  3|4.97|
|1981|  4|6.46|
|1981|  5|5.56|
+----+---+----+
only showing top 20 rows



In [0]:
# TOTALIZANDO O PERCENTUAL DO IPCA POR ANO E GERANDO O GRÁFICO COM DISPLAY

dfipca_ano = dfipca.groupby('ano').sum('perc').sort('ano', ascending=False)

from pyspark.sql.functions import round, col
dfipca_ano = dfipca_ano.select('ano', round(col('sum(perc)'), 2).alias('perc'))

dfipca_ano.select('*').show()
display(dfipca_ano)

+----+-----+
| ano| perc|
+----+-----+
|2022| 4.69|
|2021| 9.63|
|2020| 4.44|
|2019| 4.23|
|2018| 3.69|
|2017| 2.91|
|2016| 6.12|
|2015|10.19|
|2014| 6.23|
|2013| 5.76|
|2012| 5.69|
|2011| 6.32|
|2010| 5.76|
|2009| 4.23|
|2008| 5.75|
|2007| 4.37|
|2006|  3.1|
|2005| 5.55|
|2004| 7.35|
|2003| 8.95|
+----+-----+
only showing top 20 rows



ano,perc
2022,4.69
2021,9.63
2020,4.44
2019,4.23
2018,3.69
2017,2.91
2016,6.12
2015,10.19
2014,6.23
2013,5.76


In [0]:
# CONVERTENDO DATAFRAME SPARK PARA DATAFRAM PANDAS GERANDO O GRÁFICO COM DISPLAY

dfPandas = dfipca_ano.toPandas()
display(dfPandas)

ano,perc
2022,4.69
2021,9.63
2020,4.44
2019,4.23
2018,3.69
2017,2.91
2016,6.12
2015,10.19
2014,6.23
2013,5.76


In [0]:
''' 
EXTRAINDO OS DADOS DE PMC(pesquisa mensal de comércio por UF - ibge - percentual de crescimento das vendas no varejo), 
DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK
'''

dfpmcUF = spark.sql("SELECT uf, ano, mes, avg(m3mensal) as perc \
                            FROM parquet.`s3a://arquivos-pmc-processeds3-uf/` \
                            where mes < 13 \
                            group by uf, ano, mes order by uf, ano, mes")

from pyspark.sql.functions import lit
#dfpmcUF = dfpmcUF.withColumn("id", lit("uf"))
dfpmcUF.show(99999)

+-------------------+----+---+--------------------+
|                 uf| ano|mes|                perc|
+-------------------+----+---+--------------------+
|               Acre|2022| 01| -0.6250000000000002|
|               Acre|2022| 02|              18.325|
|               Acre|2022| 03|  6.8500000000000005|
|               Acre|2022| 04|              11.425|
|               Acre|2022| 05|               3.725|
|            Alagoas|2022| 01|                6.35|
|            Alagoas|2022| 02|               8.775|
|            Alagoas|2022| 03|  12.100000000000001|
|            Alagoas|2022| 04|                22.0|
|            Alagoas|2022| 05|              16.575|
|              Amapá|2022| 01|               -4.75|
|              Amapá|2022| 02|                6.35|
|              Amapá|2022| 03|  24.075000000000003|
|              Amapá|2022| 04|                10.4|
|              Amapá|2022| 05|               -3.55|
|           Amazonas|2022| 01|                38.2|
|           

In [0]:
# TOTALIZANDO O PERCENTUAL DE CRESCIMENTO DE VENDAS POR ANO E UF GERANDO O GRÁFICO COM DISPLAY

dfpmcUF_ano = dfpmcUF.groupby('ano', 'uf').sum('perc')
 
from pyspark.sql.functions import round, col
dfpmcUF_ano = dfpmcUF_ano.select('uf', 'ano', round(col('sum(perc)'), 2).alias('perc')).sort('ano', 'perc', ascending=False)
display(dfpmcUF_ano)

uf,ano,perc
Roraima,2022,87.95
Mato Grosso do Sul,2022,78.25
Mato Grosso,2022,74.88
Ceará,2022,71.85
Espírito Santo,2022,70.4
Alagoas,2022,65.8
Pará,2022,65.58
Santa Catarina,2022,64.95
Amazonas,2022,60.98
Rio Grande do Sul,2022,60.35


In [0]:
''' 
EXTRAINDO OS DADOS DE PMC(pesquisa mensal de comércio por categoria de comércio - ibge - percentual de crescimento das vendas no varejo), 
DOS ARQUIVOS PARQUET PARA O DATAFRAME PYSPARK
'''

dfpmcCLCOM = spark.sql("SELECT classe_comercio as classe, ano, mes, avg(m3mensal) as perc \
                            FROM parquet.`s3a://arquivos-pmc-processeds3-catcom/` \
                            where mes < 13 and m3mensal > 0 \
                            group by classe_comercio, ano, mes order by classe_comercio, ano, mes")

from pyspark.sql.functions import lit
#dfpmcCLCOM = dfpmcCLCOM.withColumn("id", lit("clcom"))
dfpmcCLCOM.show(99999)

+--------------------+----+---+------------------+
|              classe| ano|mes|              perc|
+--------------------+----+---+------------------+
|1. Combustíveis e...|2022| 01| 82.93333333333334|
|1. Combustíveis e...|2022| 02| 88.38424666666667|
|1. Combustíveis e...|2022| 03|        65.6191825|
|1. Combustíveis e...|2022| 04|        68.4109275|
|1. Combustíveis e...|2022| 05|        68.1745075|
|10. Material de c...|2022| 01| 97.63333333333333|
|10. Material de c...|2022| 02|102.06112333333333|
|10. Material de c...|2022| 03| 78.32158749999999|
|10. Material de c...|2022| 04| 98.05826333333334|
|10. Material de c...|2022| 05| 98.09613666666667|
|2. Hipermercados,...|2022| 01|              86.2|
|2. Hipermercados,...|2022| 02|        70.7502325|
|2. Hipermercados,...|2022| 03| 90.82128333333333|
|2. Hipermercados,...|2022| 04|        70.1241625|
|2. Hipermercados,...|2022| 05| 96.23495333333334|
|2.1. Hipermercado...|2022| 01| 87.76666666666667|
|2.1. Hipermercado...|2022| 02|

In [0]:
# TOTALIZANDO O PERCENTUAL DE CRESCIMENTO DE VENDAS POR ANO E CLASSE DE COMÉRCIO GERANDO O GRÁFICO COM DISPLAY

dfpmcCLCOM_ano = dfpmcCLCOM.groupby('ano', 'classe').sum('perc')
 
from pyspark.sql.functions import round, col
dfpmcCLCOM_ano = dfpmcCLCOM_ano.select('classe', 'ano', round(col('sum(perc)'), 2).alias('perc')).sort('ano', 'perc', ascending=False)
display(dfpmcCLCOM_ano)

classe,ano,perc
Comércio Varejista Ampliado (2),2022,604.12
10. Material de construção,2022,474.17
"5. Artigos farmacêuticos, médicos, ortopédicos, de perfumaria e cosméticos",2022,435.29
8. Outros artigos de uso pessoal e doméstico,2022,420.8
2.1. Hipermercados e supermercados,2022,420.66
"2. Hipermercados, supermercados, produtos alimentícios bebidas e fumo",2022,414.13
4. Móveis e eletrodomésticos,2022,401.58
4.2. Eletrodomésticos,2022,399.06
1. Combustíveis e lubrificantes,2022,373.52
"9. Veículos, motocicletas, partes e peças",2022,333.21
